In [1]:
import numpy as np
import pandas as pd
import sys
import gc
sys.path.append("../src")
from config import read_config, update_config
from metric import cal_mae_metric
from dataset import read_data
from FE import add_features_choice
from util import smart_avg,reduce_mem_usage

In [2]:
config = read_config("LSTM5_CLS_DO02_CH01_IPOnly_SiLU_ADAM_PL")
config.gpu = [0]
config.debug = True
config = update_config(config)

Using device: cuda
Number of device: 1
Model Output Folder: /home/vincent/Kaggle/GBVPP/output/LSTM5_CLS_DO02_CH01_IPOnly_SiLU_ADAM_PL/


In [3]:
train, test = read_data(config)
train.drop(columns="pressure", inplace=True)
all_data = pd.concat([train, test],axis=0).reset_inde
del train, test
gc.collect()

0

In [4]:
all_data.shape

(204800, 8)

In [5]:
all_data = add_features_choice(all_data.copy(), config)
all_data, NAlist = reduce_mem_usage(all_data)

--- Generate u_out features ---


ValueError: cannot reindex from a duplicate axis